In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import numpy as np
import logging
import os
import matplotlib.pyplot as plt
import pickle
from transformers import ViTFeatureExtractor, ViTModel
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from scipy.special import softmax

IMG_SHAPE = 224

import torch
import clip
print(torch.__version__)
print("torch detects GPU: " , torch.cuda.is_available())

1.12.1+cu116
torch detects GPU:  True


# Resizing the images

In [2]:
# load every image in the food folder and resize them to 224x224, then save them in a new folder

# for filename in os.listdir('food'):
#     img = Image.open('food/' + filename)
#     img = img.resize((IMG_SHAPE, IMG_SHAPE))
#     img.save('food_'+str(IMG_SHAPE)+'/' + filename)


# Extracting the features

In [6]:
image = Image.open('food_'+str(IMG_SHAPE)+'/00007.jpg')

feature_extractor = AutoFeatureExtractor.from_pretrained("nateraw/food")
model = AutoModelForImageClassification.from_pretrained("nateraw/food")

# feature_extractor = AutoFeatureExtractor.from_pretrained("aspis/swin-finetuned-food101")
# model = AutoModelForImageClassification.from_pretrained("aspis/swin-finetuned-food101")

#put the models on the GPU
# feature_extractor = feature_extractor.to('cuda')
model = model.to('cuda')
# print(model)

#remove the classifier layer
model.classifier = torch.nn.Identity()
# model.vit.layernorm = torch.nn.Identity()
# print(model)

with torch.no_grad():
    np.set_printoptions(suppress=True)
    inputs = feature_extractor(images=image, return_tensors="pt").to('cuda')
    # print(inputs)
    outputs = model(**inputs)
    # soft = outputs.logits.softmax(dim=-1).cpu().numpy()
    # print(np.argmax(soft))
    # print(outputs)
    print(outputs.logits.cpu().numpy().shape)
    print(outputs.logits.cpu().numpy())



(1, 1024)
[[-0.9790746   2.6313405  -0.21268739 ...  0.3248024  -0.08107211
   0.30868477]]


In [10]:
# dictionary with all the image names and their features
features = {}

# convert every image into features
line_id = 0
for filename in os.listdir('food_'+str(IMG_SHAPE)+''):
    id = int(filename.replace(".jpg", ""))
    # print(id)
    line_id += 1
    image = Image.open('food_'+str(IMG_SHAPE)+'/' + filename)
    with torch.no_grad():
        inputs = feature_extractor(images=image, return_tensors="pt").to('cuda')
        feature = model(**inputs).logits
        feature = np.array(feature.cpu().numpy(), dtype=np.float32)
        feature = feature.reshape(-1)
        features[filename.replace(".jpg","")] = feature
        # print(feature[0])
        if line_id % 100 == 0:
            print(line_id)
            # break
        if id == 5001:
            break

# save the features dictionary to a file using pickle
with open('features_vit.pickle', 'wb') as handle:
# with open('features_swin.pickle', 'wb') as handle:
    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [11]:

# features = {}
# with open('features_mn.pickle', 'rb') as handle:
#     features = pickle.load(handle)
    
print(features['00000'].shape)
print(features['00000'])
print(np.array(features['00000'][0], dtype=np.float32))
print(np.array(features['00001'][0], dtype=np.float32))
print(np.array(features['00002'][0], dtype=np.float32))
print(len(features))

(1024,)
[ 1.6176932 -0.3299582 -0.1449341 ... -1.4465858  2.7391245 -1.6128058]
1.6176932
2.0003452
-2.1996455
5002


In [ ]:
# features = {}

# with open('features_vit2.pickle', 'rb') as handle:
#     features = pickle.load(handle)
#     for key in features:
#         features[key] = softmax(features[key])
#     # print(np.array(features['00000'], dtype=np.float32))
#     # print(np.array(features['00001'][0], dtype=np.float32))
#     # print(np.array(features['00002'][0], dtype=np.float32))


# with open('features_vit3.pickle', 'wb') as handle:
#     pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)